<a href="https://colab.research.google.com/github/MrSimple07/MachineLearning_Practice/blob/main/KMU_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Roberta

In [4]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model_name = "bert-base-uncased"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [17]:
from sklearn.metrics import f1_score

prompts = [
    {'question': 'Продолжение этого фильма Евгения Матвеева называется "Судьба".', 'context': 'Любовь земная'},
    {'question': 'С ЭТИМ ЧЕЛОВЕКОМ, 33-м потомком Нина, Державин сравнивает богатого и ленивого вельможу.', 'context': 'Сарданапал (Нин, как и Сарданапал, — легендарный правитель Ассирии)'},
    {'question': 'По мнению древнего мудреца Рюхейб-бин-Верда: "Мудрость состоит из 10 частей. Одна часть — знания и опыт." А что составляет остальные девять?', 'context': 'Молчание'},
    {'question': 'По одной из версий, начало названия ЭТОЙ ОРГАНИЗАЦИИ происходит от греческого слова, обозначающего круг.', 'context': 'Ку-клукс-клан (греч. "киклос")'},
    {'question': 'На девятерной вистуют только эти 2 категории людей', 'context': 'Попы и студенты (попы от жадности, а студенты — от бедности :-))'},
    {'question': 'Эпиграф к книге "Корпорация М.И.Ф." Асприна звучит так: "Главное — не злоупотреблять...". Чем?', 'context': 'Эпиграфами'},
    {'question': 'Кроме известных исторических полотен, ЕГО кисти принадлежат росписи "первой версии" Храма Христа Спасителя.', 'context': 'Василий Суриков'},
    {'question': 'Именно столько морей омывает берега Италии.', 'context': 'Пять (Ионическое, Тирренское, Лигурийское, Адриатическое и Средиземное)'},
    {'question': 'Именно ему Эразм Роттердамский посвятил свою книгу "Похвала глупости", а церковь канонизировала его в 1935 году, хотя он и написал книгу о "месте, которого нет".', 'context': 'Томас Мор ("Утопия")'},
]

# Check accuracy
correct_predictions = 0


correct_predictions = 0
total_prompts = len(prompts)

for prompt in prompts:
    response = nlp(prompt)
    predicted_answer = response['answer']

    # Compare predicted answer with the expected answer
    expected_answer = prompt['context']
    if predicted_answer == expected_answer:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_prompts

# True answers (ground truth)
true_answers = [prompt['context'] for prompt in prompts]

# Predicted answers
predicted_answers = [nlp(prompt['question'], prompt['context'])['answer'] for prompt in prompts]

# Convert answers to binary (1 if correct, 0 if incorrect)
binary_true_answers = [1 if true == pred else 0 for true, pred in zip(true_answers, predicted_answers)]

# Calculate F1 score
f1 = f1_score(binary_true_answers, [1] * len(binary_true_answers))  # Binary F1, assuming all answers are correct

# Print F1 score
print(f"F1 Score: {f1:.2f}")

# Print accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

F1 Score: 0.50
Accuracy: 33.33%


In [20]:
prompt = {
    'text': 'Продолжение этого фильма Евгения Матвеева называется "Судьба".',
    'topic': 'Любовь'
}

expected_answer = "Любовь земная"

# Evaluate the model on the prompt
model_answer = nlp(prompt['text'], prompt['text'])['answer']

# Calculate accuracy
accuracy = 1 if model_answer == expected_answer else 0


print(f"Prompt: {prompt['text']}")
print(f"Expected Answer: {expected_answer}")
print(f"Model Answer: {model_answer}")
print(f"Accuracy: {accuracy}")


Prompt: Продолжение этого фильма Евгения Матвеева называется "Судьба".
Expected Answer: Любовь земная
Model Answer: Матвеева
Accuracy: 0
